In [1]:
import pandas as pd
import ccxt
from statsmodels.regression.rolling import RollingOLS
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint

plt.style.use("seaborn-v0_8")

In [2]:
def fetch_prices(binance, symbol_one, symbol_two, look_back_days): 
    # Prepare empty dataframe for results
    df_final = pd.DataFrame()
    
    # Calculate the end timestamp for the data fetching
    end_timestamp = binance.milliseconds()
    since_timestamp = end_timestamp - look_back_days * 24 * 60 * 60 * 1000 
    
    while since_timestamp < end_timestamp:
        try:
            # Fetch OHLCV data
            ohlcv_theta = binance.fetch_ohlcv(symbol_two, '1h', since=since_timestamp, limit=1000)
            ohlcv_xrp = binance.fetch_ohlcv(symbol_one, '1h', since=since_timestamp, limit=1000)
            
            # Create temporary dataframes to hold fetched data
            df_theta_temp = pd.DataFrame(ohlcv_theta, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df_xrp_temp = pd.DataFrame(ohlcv_xrp, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

            # Check if both dataframes have the same length
            if len(df_theta_temp) != len(df_xrp_temp):
                print(
                    f"Data length mismatch: {symbol_two} has {len(df_theta_temp)} rows, {symbol_one} has {len(df_xrp_temp)} rows.")
                return None
            
            # Merge dataframes
            df_temp = pd.merge(df_xrp_temp[['timestamp', 'close']], df_theta_temp[['timestamp', 'close']], on='timestamp', how='inner')
            df_temp.columns = ['timestamp', symbol_one, symbol_two]
            
            # Concatenate with the final dataframe
            df_final = pd.concat([df_final, df_temp])
            
            # Update the since_timestamp to the last timestamp fetched
            since_timestamp = df_final['timestamp'].max() + (60 * 60 * 1000)  # Add one hour

        except Exception as e:
            print(f"Error in fetching prices: {e}")
            break
    
    # Remove duplicates and sort the dataframe
    df_final.drop_duplicates(subset='timestamp', keep='first', inplace=True)
    df_final.sort_values(by='timestamp', inplace=True)
    
    return df_final


symbol_one = "ATOMUSDT"
symbol_two = "AXSUSDT"

binance = ccxt.binance()

prices = fetch_prices(binance, symbol_one, symbol_two, 15)

x = prices[symbol_one]
y = prices[symbol_two]

model = sm.OLS(y, sm.add_constant(x)).fit()
hedge_ratio = model.params[1]
print("Hedge Ratio: ", hedge_ratio)


Hedge Ratio:  0.4020663256848915
